In [1]:
import pandas as pd
import os

# 1. Concatenar os arquivos anuais de 1992 a 2024
matches_list = []

for year in range(1992, 2025):
    file_path = f"D:/projetos/Tenis ML-AI/data/tennis_atp/atp_matches_{year}.csv"
    
    # Verificar se o arquivo existe antes de carregar
    if os.path.exists(file_path):
        df_year = pd.read_csv(file_path)
        matches_list.append(df_year)
        print(f"Arquivo {year} carregado com {len(df_year)} registros")
    else:
        print(f"Arquivo {year} não encontrado")

# Concatenar todos os DataFrames
matches = pd.concat(matches_list, ignore_index=True)
print(f"Dataset completo com {len(matches)} partidas de 1992 a 2024")

Arquivo 1992 carregado com 3792 registros
Arquivo 1993 carregado com 3890 registros
Arquivo 1994 carregado com 3938 registros
Arquivo 1995 carregado com 3800 registros
Arquivo 1996 carregado com 3774 registros
Arquivo 1997 carregado com 3623 registros
Arquivo 1998 carregado com 3591 registros
Arquivo 1999 carregado com 3334 registros
Arquivo 2000 carregado com 3378 registros
Arquivo 2001 carregado com 3307 registros
Arquivo 2002 carregado com 3213 registros
Arquivo 2003 carregado com 3218 registros
Arquivo 2004 carregado com 3288 registros
Arquivo 2005 carregado com 3264 registros
Arquivo 2006 carregado com 3267 registros
Arquivo 2007 carregado com 3192 registros
Arquivo 2008 carregado com 3123 registros
Arquivo 2009 carregado com 3085 registros
Arquivo 2010 carregado com 3030 registros
Arquivo 2011 carregado com 3015 registros
Arquivo 2012 carregado com 3009 registros
Arquivo 2013 carregado com 2944 registros
Arquivo 2014 carregado com 2901 registros
Arquivo 2015 carregado com 2943 re

In [4]:
# 2. Carregar o dataset de jogadores com ELO
players = pd.read_csv('D:/projetos/Tenis ML-AI/src/players_with_updated_elo.csv')

# 3. Pré-processamento dos dados
# Limpeza de nomes nos datasets
matches['winner_name'] = matches['winner_name'].str.strip()
matches['loser_name'] = matches['loser_name'].str.strip()

# Criar coluna de nome completo formatado
matches['winner_full_name'] = matches['winner_name'].str.split().str.join(' ')
matches['loser_full_name'] = matches['loser_name'].str.split().str.join(' ')

In [5]:
# Processar o dataset de jogadores
players = players.rename(columns={
    'player_id': 'id', 
    'name_first': 'first_name', 
    'name_last': 'last_name'
})

players['full_name'] = (players['first_name'] + ' ' + players['last_name']).str.strip()

# 4. Merge com os dados de ELO - Vencedores
matches = pd.merge(
    matches,
    players[['id', 'full_name', 'current_elo', 'peak_elo']].rename(columns={
        'current_elo': 'winner_current_elo',
        'peak_elo': 'winner_peak_elo'
    }),
    left_on=['winner_id', 'winner_full_name'],
    right_on=['id', 'full_name'],
    how='left'
).drop(['id', 'full_name'], axis=1)


In [6]:
# 5. Merge com os dados de ELO - Perdedores
matches = pd.merge(
    matches,
    players[['id', 'full_name', 'current_elo', 'peak_elo']].rename(columns={
        'current_elo': 'loser_current_elo',
        'peak_elo': 'loser_peak_elo'
    }),
    left_on=['loser_id', 'loser_full_name'],
    right_on=['id', 'full_name'],
    how='left'
).drop(['id', 'full_name'], axis=1)

# 6. Remover colunas temporárias
matches = matches.drop(['winner_full_name', 'loser_full_name'], axis=1)

In [7]:
# 7. Verificar resultados
print("\nInformações do dataset combinado:")
print(matches.info())
print("\nPrimeiras linhas:")
print(matches.head())


Informações do dataset combinado:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104648 entries, 0 to 104647
Data columns (total 53 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   tourney_id          104648 non-null  object 
 1   tourney_name        104648 non-null  object 
 2   surface             104595 non-null  object 
 3   draw_size           104648 non-null  int64  
 4   tourney_level       104648 non-null  object 
 5   tourney_date        104648 non-null  int64  
 6   match_num           104648 non-null  int64  
 7   winner_id           104648 non-null  int64  
 8   winner_seed         42514 non-null   float64
 9   winner_entry        12897 non-null   object 
 10  winner_name         104648 non-null  object 
 11  winner_hand         104648 non-null  object 
 12  winner_ht           102515 non-null  float64
 13  winner_ioc          104648 non-null  object 
 14  winner_age          104643 non-null  float64
 15 

In [8]:
# 8. Salvar o resultado (opcional)
output_path = "D:/projetos/Tenis ML-AI/notebooks/atp_matches_1992_2024_with_elo.csv"
matches.to_csv(output_path, index=False)
print(f"\nDataset combinado salvo em: {output_path}")


Dataset combinado salvo em: D:/projetos/Tenis ML-AI/notebooks/atp_matches_1992_2024_with_elo.csv
